#### [ 임베딩 이해 ]

In [18]:
## => Embedding  : 고차원 피쳐 ==> 저차원 
## => Pytorch    : Embedding Layer 제공 
##    * 고정 길이 : Embedding    Layer
##    * 동적 길이 : EmbeddingBag Layer
## => 사전 학습된 성능 좋은 Embeddign Model들 존재
##    : Word2Vec, Doc2Vec, FastText, ..., CBOW, Skip-gram...

In [19]:
## 모듈 로딩
import torch
import torch.nn as nn
import torch.optim as optim

In [20]:
## 임의의 데이터 => 토큰화 
sentence = "Repeat is the best medicine for memory".split()

In [21]:
## 데이터기반의 단어사전 생성 [1] 단어 추출
vocab = list(set(sentence))
print(vocab)

['the', 'Repeat', 'for', 'medicine', 'best', 'memory', 'is']


In [22]:
## 데이터기반의 단어사전 생성 [2] 단어별 정수 부여 
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0


In [23]:
print(word2index)
print(word2index['memory'])

{'the': 1, 'Repeat': 2, 'for': 3, 'medicine': 4, 'best': 5, 'memory': 6, 'is': 7, '<unk>': 0}
6


In [24]:
# 숫자 => 토큰/반환  사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'the', 2: 'Repeat', 3: 'for', 4: 'medicine', 5: 'best', 6: 'memory', 7: 'is', 0: '<unk>'}


In [25]:
print(index2word[2])

Repeat


In [26]:
## 모든 문장 => 수치 벡터화 & 텐서화 진행
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence]     # 각 문자를 정수로 변환

    input_seq,  label_seq = encoded[:-1], encoded[1:]        # 입력 시퀀스와 레이블 시퀀스를 분리
    print(f'{input_seq} => {label_seq}')
    input_seq = torch.LongTensor(input_seq).unsqueeze(0)     # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0)     # 배치 차원 추가
    
    return input_seq, label_seq

In [27]:
X, Y = build_data(sentence, word2index)

[2, 7, 1, 5, 4, 3] => [7, 1, 5, 4, 3, 6]


- 모델 정의 및 설계<hr>

In [28]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        # 워드 임베딩
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, 
                                            embedding_dim=input_size)
        
        # 입력 차원, 은닉 상태의 크기 정의
        self.rnn_layer = nn.RNN(input_size, hidden_size, 
                                batch_first=batch_first)
        
        # 출력 : 원-핫 벡터 크기 또는 단어 집합 크기
        self.linear = nn.Linear(hidden_size, vocab_size) 
        
    def forward(self, x):
        # 1. 임베딩 층
        #    크기 변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        #    크기 변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        #    => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        #    크기 변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        #    크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

- 학습 관련 준비<hr>

In [29]:
# 하이퍼 파라미터
vocab_size  = len(word2index)   # vocab 크기는 임베딩 층, 최종 출력층에 사용
                                # <unk> 토큰을 크기에 포함한다.
input_size  = 5                 # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20                # RNN의 은닉층 크기

In [30]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [31]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)

tensor([[-0.2823,  0.2066,  0.1922,  0.3079,  0.1735, -0.2066, -0.2854, -0.1772],
        [-0.0888, -0.0298,  0.1072,  0.3036, -0.1249, -0.3878, -0.1253, -0.1661],
        [-0.6452,  0.4686,  0.4161,  0.4939,  0.1358, -0.3801, -0.3319, -0.3985],
        [-0.3287,  0.4340,  0.3679,  0.2452,  0.3344, -0.1996,  0.0576, -0.4304],
        [-0.1137,  0.0960,  0.3885,  0.3475,  0.1399, -0.0059,  0.1530, -0.3874],
        [-0.4978, -0.2345,  0.3951,  0.6351, -0.4855, -0.3147,  0.1773, -0.2902]],
       grad_fn=<ViewBackward0>)


In [32]:
print(output.shape)

torch.Size([6, 8])


In [33]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [34]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    print("[{:02d}/201] {:.4f} ".format(step+1, loss))
    
    pred = output.softmax(-1).argmax(-1).tolist()
    print(" ".join(["Repeat"] + decode(pred)))
    print()

[01/201] 2.0760 
Repeat for for for the Repeat for

[02/201] 2.0598 
Repeat for for for the Repeat for

[03/201] 2.0436 
Repeat for for for the Repeat for

[04/201] 2.0276 
Repeat for for for the for for

[05/201] 2.0116 
Repeat for for for the for for

[06/201] 1.9956 
Repeat for for for the for for

[07/201] 1.9797 
Repeat for for for medicine for for

[08/201] 1.9639 
Repeat for for for medicine for for

[09/201] 1.9481 
Repeat for for for medicine for for

[10/201] 1.9324 
Repeat for for for medicine for for

[11/201] 1.9167 
Repeat for for for medicine for for

[12/201] 1.9011 
Repeat for for for medicine for for

[13/201] 1.8854 
Repeat for for for medicine for for

[14/201] 1.8699 
Repeat for for for medicine for for

[15/201] 1.8543 
Repeat for for for medicine for for

[16/201] 1.8388 
Repeat for for for medicine for for

[17/201] 1.8232 
Repeat for for the medicine for for

[18/201] 1.8077 
Repeat for for the medicine for for

[19/201] 1.7922 
Repeat for the the medicine for 